# 1. Profilování a benchmarking

V této části si ukážeme, jak měřit výkon kódu a jak hledat úzká místa.

- **Profilování** ukáže, kde program tráví čas.
- **Benchmarking** porovnává různé implementace na stejné sadě vstupů.

Typický postup je:
1. najít problémová místa profilováním,
2. upravit algoritmus nebo implementaci,
3. ověřit přínos benchmarkem.

## 1.1 Co sledovat při profilování

V Pythonu existuje více profilerů a každý měří jiný pohled na výkon:
- čas funkcí nebo jednotlivých řádků,
- paměťové nároky,
- případně i GPU část výpočtu.

Profilery běh zpomalují, proto berte absolutní časy orientačně. Důležité jsou hlavně poměry mezi částmi kódu a mezi variantami řešení.

## 1.2 Ukázkový kód pro měření

Ukázka je záměrně parametrizovaná (`scale`), aby šlo snadno měnit velikost workloadu bez přepisování funkcí.

In [ ]:
import numpy as np


def build_profile_config(scale=1):
    return {
        "batch_count": 12,
        "rows": 550,
        "cols": 550,
        "repeat_factor": 3,
        "selected_fraction": 0.5,
        "hotspot_fraction": 1.0,
        "keep_every": 5,
        "seed": 42,
        "rounds": max(1, int(scale)),
    }


def prepare_batches(batch_count, rows, cols, seed):
    rng = np.random.default_rng(seed)
    return [rng.random((rows, cols), dtype=np.float64) for _ in range(batch_count)]


def compute_signal(matrix):
    centered = matrix - matrix.mean(axis=1, keepdims=True)
    scaled = centered / (matrix.std(axis=1, keepdims=True) + 1e-9)
    transformed = np.sin(scaled) + np.sqrt(np.abs(scaled) + 1.0)
    return transformed


def python_hotspot(matrix, hotspot_width):
    row_scores = []
    for row in matrix:
        acc = 0.0
        for value in row[:hotspot_width]:
            acc += (value * value) / (1.0 + value)
        row_scores.append(acc)
    return row_scores


def process_batches(batches, selected_fraction, repeat_factor, hotspot_fraction, keep_every):
    kept_arrays = []
    batch_scores = []
    for batch_index, batch in enumerate(batches):
        features = compute_signal(batch)
        selected_columns = max(16, int(features.shape[1] * selected_fraction))
        expanded = np.repeat(features[:, :selected_columns], repeat_factor, axis=1).copy()

        hotspot_width = max(16, int(expanded.shape[1] * hotspot_fraction))
        row_scores = python_hotspot(expanded, hotspot_width)
        batch_scores.append(float(np.mean(row_scores)))

        if batch_index % keep_every == 0:
            kept_arrays.append(expanded)

    return np.array(batch_scores), kept_arrays


def code_setup(scale=1):
    cfg = build_profile_config(scale)
    print(f"Preparing data for scale={cfg['rounds']}")
    batches = prepare_batches(
        batch_count=cfg["batch_count"],
        rows=cfg["rows"],
        cols=cfg["cols"],
        seed=cfg["seed"],
    )
    print("Processing batches")

    all_scores = []
    history = []
    for _ in range(cfg["rounds"]):
        scores, kept = process_batches(
            batches,
            selected_fraction=cfg["selected_fraction"],
            repeat_factor=cfg["repeat_factor"],
            hotspot_fraction=cfg["hotspot_fraction"],
            keep_every=cfg["keep_every"],
        )
        all_scores.append(scores)
        history.extend(kept)

    combined_scores = np.concatenate(all_scores)
    print(f"Done: {len(combined_scores)} score items, kept {len(history)} arrays")
    return combined_scores, history


## 1.3 Základní profilování: `cProfile`

`cProfile` je vestavěný profiler ze standardní knihovny. Měří volání funkcí a jejich trvání, ale neprofiluje jednotlivé řádky.

V základní podobě stačí použít `cProfile.run()`, předat volání profilované funkce a název výstupního `.prof` souboru.

In [ ]:
import cProfile

cProfile.run("code_setup(scale=2)", "vysledky_cProfile_code_setup.prof")

Výsledky lze zobrazit i vizuálně přes `snakeviz` (`pip install snakeviz`). Otevře interaktivní stránku s grafem volání a tabulkou funkcí.

In [ ]:
# `snakeviz` obvykle otevřete z terminálu příkazem:
# snakeviz vysledky_cProfile_code_setup.prof

Výsledky lze číst také jako text pomocí modulu `pstats`.

- `p = pstats.Stats("soubor.prof")` načte profil.
- `.strip_dirs()` zkrátí cesty k funkcím.
- `.sort_stats("cumulative")` seřadí funkce podle kumulativního času.
- `.print_stats(12)` vypíše nejdražší záznamy.

In [ ]:
import pstats

p = pstats.Stats("vysledky_cProfile_code_setup.prof")
p.strip_dirs().sort_stats("cumulative").print_stats(12)
print("\nPouze funkce z ukázky:")
p.print_stats("code_setup|prepare_batches|process_batches|compute_signal|python_hotspot")


Stejný postup jde v prostředí Jupyter notebooků provést pohodlně i přes magic příkaz `%prun`:

In [ ]:
%prun -s cumulative -l 12 code_setup(scale=2)

## 1.4 Řádkové profilování: `line_profiler`

Když nestačí profil po funkcích, hodí se `line_profiler`, který měří čas po jednotlivých řádcích.

`line_profiler` můžeme v notebooku spustit i přes Python API, které vrací přehled po řádcích přímo do výstupu buňky.

In [ ]:
from line_profiler import LineProfiler

Profilujeme funkce `compute_signal`, `python_hotspot` a `process_batches`.
Ve výstupu sledujte hlavně řádky s nejvyšším podílem času.

In [ ]:
cfg = build_profile_config(scale=1)
batches = prepare_batches(
    batch_count=cfg["batch_count"],
    rows=cfg["rows"],
    cols=cfg["cols"],
    seed=cfg["seed"],
)

lp = LineProfiler(compute_signal, python_hotspot, process_batches)
lp.runcall(
    process_batches,
    batches,
    selected_fraction=cfg["selected_fraction"],
    repeat_factor=cfg["repeat_factor"],
    hotspot_fraction=cfg["hotspot_fraction"],
    keep_every=cfg["keep_every"],
)
lp.print_stats(output_unit=1e-6)


## 1.5 Kombinované profilování: `scalene`

`scalene` kombinuje profilování funkcí, řádků i paměti v jednom nástroji.

Instalace: `pip install scalene`

V této lekci použijeme jednoduchý postup:
1. `!scalene run -o scalene-profile.json code_profile_test.py`
2. `!scalene view --standalone scalene-profile.json`

Druhý příkaz vytvoří soubor `scalene-profile.html`.

In [ ]:
%%writefile code_profile_test.py
import numpy as np


def build_profile_config(scale=1):
    return {
        "batch_count": 12,
        "rows": 550,
        "cols": 550,
        "repeat_factor": 3,
        "selected_fraction": 0.5,
        "hotspot_fraction": 1.0,
        "keep_every": 5,
        "seed": 42,
        "rounds": max(1, int(scale)),
    }


def prepare_batches(batch_count, rows, cols, seed):
    rng = np.random.default_rng(seed)
    return [rng.random((rows, cols), dtype=np.float64) for _ in range(batch_count)]


def compute_signal(matrix):
    centered = matrix - matrix.mean(axis=1, keepdims=True)
    scaled = centered / (matrix.std(axis=1, keepdims=True) + 1e-9)
    transformed = np.sin(scaled) + np.sqrt(np.abs(scaled) + 1.0)
    return transformed


def python_hotspot(matrix, hotspot_width):
    row_scores = []
    for row in matrix:
        acc = 0.0
        for value in row[:hotspot_width]:
            acc += (value * value) / (1.0 + value)
        row_scores.append(acc)
    return row_scores


def process_batches(batches, selected_fraction, repeat_factor, hotspot_fraction, keep_every):
    kept_arrays = []
    batch_scores = []
    for batch_index, batch in enumerate(batches):
        features = compute_signal(batch)
        selected_columns = max(16, int(features.shape[1] * selected_fraction))
        expanded = np.repeat(features[:, :selected_columns], repeat_factor, axis=1).copy()

        hotspot_width = max(16, int(expanded.shape[1] * hotspot_fraction))
        row_scores = python_hotspot(expanded, hotspot_width)
        batch_scores.append(float(np.mean(row_scores)))

        if batch_index % keep_every == 0:
            kept_arrays.append(expanded)

    return np.array(batch_scores), kept_arrays


def code_setup(scale=1):
    cfg = build_profile_config(scale)
    print(f"Preparing data for scale={cfg['rounds']}")
    batches = prepare_batches(
        batch_count=cfg["batch_count"],
        rows=cfg["rows"],
        cols=cfg["cols"],
        seed=cfg["seed"],
    )
    print("Processing batches")

    all_scores = []
    history = []
    for _ in range(cfg["rounds"]):
        scores, kept = process_batches(
            batches,
            selected_fraction=cfg["selected_fraction"],
            repeat_factor=cfg["repeat_factor"],
            hotspot_fraction=cfg["hotspot_fraction"],
            keep_every=cfg["keep_every"],
        )
        all_scores.append(scores)
        history.extend(kept)

    combined_scores = np.concatenate(all_scores)
    print(f"Done: {len(combined_scores)} score items, kept {len(history)} arrays")
    return combined_scores, history


if __name__ == "__main__":
    code_setup(scale=10)


In [ ]:
!scalene run -o scalene-profile.json code_profile_test.py

In [ ]:
!scalene view --standalone scalene-profile.json

Vygenerovaný soubor `scalene-profile.html` otevřete přímo ve VS Code (Preview) nebo v prohlížeči.

V reportu sledujte hlavně:
- Python-CPU hotspot na řádcích smyčky `for value in row[:hotspot_width]`,
- NumPy výpočet v řádku `np.sin(...) + np.sqrt(...)`,
- kopírování paměti na řádku `np.repeat(...).copy()`.